# 作业——下载自己县的空间数据并划分3-5类土地利用类型

In [ ]:
## 将县域边界上传到gee中
# 参考：https://zhuanlan.zhihu.com/p/29118911

# 将 Shapefile 上传到 Google Cloud Storage
# 首先，你需要将 Shapefile 上传到 Google Cloud Storage。你可以使用 gsutil 命令行工具或者 Google Cloud Console 来完成这一步。

# bash
# gsutil cp your_shapefile.shp gs://your-bucket-name/path/to/your_shapefile.shp
# gsutil cp your_shapefile.shx gs://your-bucket-name/path/to/your_shapefile.shx
# gsutil cp your_shapefile.dbf gs://your-bucket-name/path/to/your_shapefile.dbf
# # 如果你的 Shapefile 包含其他文件（如 .prj, .qix），也一并上传

In [1]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-hyf7753')

In [2]:
## 获取安岳县的矢量图层
China_boundaries = ee.FeatureCollection('projects/ee-hyf7753/assets/2023_china_county_boundaries')
anyue_boundaries = China_boundaries.filterMetadata('区划名','equals','安岳县')

In [ ]:
## 可视化获取到的矢量图层
m = geemap.Map()
m.add_layer(anyue_boundaries, {}, 'anyue_boundaries')
m

In [3]:
## 获取安岳县栅格图层
# 从COPERNICUS/S2_SR_HARMONIZED集合中筛选位于目标区域、时间范围内且云量小于80%的影像

start_time = '2022-06-01'
end_time = '2022-10-15'

anyue_image = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(anyue_boundaries)
    .filterDate(start_time, end_time)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .sort('system:time_start',False)

)
print('S2：',anyue_image)

mosaic = anyue_image.mosaic().clip(anyue_boundaries)

S2： ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.limit",
    "arguments": {
      "ascending": {
        "constantValue": false
      },
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "ImageCollection.load",
                            "arguments": {
                              "id": {
                                "constantValue": "COPERNICUS/S2_SR_HARMONIZED"
                              }
    

In [34]:
## 可视化
visualization = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}
m = geemap.Map()
# 定位和缩放到 Polygon 所在的区域
m.centerObject(anyue_boundaries, zoom=8)  # 可以调整 zoom 级别以适合你的需求
m.add_layer(mosaic, visualization, 'RGB')
m.addLayer(anyue_boundaries, {'color': 'FF0000'}, 'Polygon Layer')
m

Map(center=[29.996204967109406, 105.39182801661268], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
geemap.download_ee_image_tiles(
    mosaic, anyue_boundaries, "anyue", prefix="s2_", crs="EPSG:3857", scale=1
)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the s2_1.tif download size (raw: 762.33 GB).


s2_1.tif: |          | 0.00/762G (raw) [  0.0%] in 00:00 (eta:     ?)

Tile download failed, retry 1 of 5.  URL: https://earthengine.googleapis.com/v1/projects/ee-hyf7753/thumbnails/a3a159e0f51b4e941748824dce61be61-2f16232ba77f380bb32ddf2418f54371:getPixels. HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/ee-hyf7753/thumbnails/a3a159e0f51b4e941748824dce61be61-2f16232ba77f380bb32ddf2418f54371:getPixels (Caused by ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))).
Tile download failed, retry 1 of 5.  URL: https://earthengine.googleapis.com/v1/projects/ee-hyf7753/thumbnails/868d11d6bae59df39b2bb1783c8dd88a-90e7db04ddfa9e0ab2a43feafdc5a772:getPixels. HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/ee-hyf7753/thumbnails/868d11d6bae59df39b2bb1783c8dd88a-90e7db04ddfa9e0ab2a43feafdc5a772:getPixels (Caused by ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现

In [ ]:
anyue_Geom = anyue_boundaries.geometry()
geemap.download_ee_image(mosaic, "anyue.tif", scale=10, region=anyue_Geom)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the anyue.tif download size (raw: 6.60 GB).


anyue.tif: |          | 0.00/6.60G (raw) [  0.0%] in 00:00 (eta:     ?)

In [44]:
anyue_Geom

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Collection.geometry",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "projects/ee-hyf7753/assets/2023_china_county_boundaries"
                  }
                }
              }
            },
            "filter": {
              "functionInvocationValue": {
                "functionName": "Filter.equals",
                "arguments": {
                  "leftField": {
                    "constantValue": "\u533a\u5212\u540d"
                  },
                  "rightValue": {
                    "constantValue": "\u5b89\u5cb3\u53bf"
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

In [ ]:
geemap.download_ee_image(m, "landsat.tif", scale=100)

In [ ]:
# 使用S2mosaic函数来合成给定时间范围内的影像，并裁剪到目标区域
def S2mosaic(start_time,end_time):
    s2 = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(anyue_boundaries).filterDate(start_time,end_time)
        .filterMetadata("CLOUD_COVERAGE_ASSESSMENT","less_than",80)
        .sort("CLOUD_COVERAGE_ASSESSMENT")
    )
    new_image = s2.mosaic().clip(anyue_boundaries)
    return new_image

In [7]:
## 日期处理：
# 使用getNewDate函数更新影像集合中每个影像的system:time_start属性，以便后续处理。

def get_new_date(image):
    time_start = image.get('system:time_start')
    date = ee.Date(time_start)
    year = date.get('year')
    month = date.get('month')
    day = date.get('day')
    new_date = ee.Date.fromYMD(year, month, day)
    image = image.set({'system:time_start': new_date.millis()})
    return image

S2_newdate = anyue_image.map(get_new_date)
print(S2_newdate)

# 使用date_str函数将日期对象转换为字符串格式
def date_str(id):
    return ee.Date(id).format('yyyy-MM-dd')

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Element.set",
              "arguments": {
                "key": {
                  "constantValue": "system:time_start"
                },
                "object": {
                  "argumentReference": "_MAPPING_VAR_0_0"
                },
                "value": {
                  "functionInvocationValue": {
                    "functionName": "Date.millis",
                    "arguments": {
                      "date": {
                        "functionInvocationValue": {
                          "functionName": "Date.fromYMD",
                          "arguments": {
                            "day": {
                              "funct

In [ ]:
function imgAdd_S2(coll){
  // var rawLayer = null;
  var computedIds_S2A = coll.reduceColumns(ee.Reducer.toList(),['system:time_start']).get('list')
  var datelist=ee.List(computedIds_S2A).distinct();
  var datelist_str=datelist.map(date_str)
  
  print(datelist_str)//有影像的时间点
  print(datelist);
  print(datelist.length());//有影像的期数和
  datelist_str.evaluate(function(ids){
    print('computedIds_S2A',ids.length);
    for(var k = 0;k < ids.length;k++){
      // print(k)
    // for(var k = ee.Number(0);k < datelist.length();k=k+1){
      var key_S2A1 = ids[k]//.slice(0,8),呈现影像时间
      // print(key_S2A1)
      var star_date = ee.Date.parse('yyyy-MM-dd',key_S2A1);
      var end_date = star_date.advance(1,'day');
      var key_S2A2 = S2mosaic(star_date,end_date).clip(region);
      // print(star_date.format("yyyy-mm-dd").serialize(false))
      Map.addLayer(key_S2A2, {bands: ['B8', 'B3', 'B2'], min:0,max:5000},key_S2A1,false)
      
  }
})
}

imgAdd_S2(S2_newdate);

In [ ]:
## 影像添加和可视化：
# 使用imgAdd_S2函数遍历筛选后的影像集合，对每个时间点合成影像，并将它们添加到地图上。

def imgAdd_S2(coll):
    computed_ids_s2a = coll.reduceColumns(ee.Reducer.toList(), ['system:time_start']).get('list')
    datelist = computed_ids_s2a.distinct()
    datelist_str = datelist.map(date_str)

    

# 单独合成并添加指定时间范围（img_time1 和 img_time2）内的目标影像（target_img）。


In [ ]:
var target_img = S2mosaic(img_time1,img_time2);
Map.addLayer(target_img, {bands: ['B8', 'B3', 'B2'], min:0,max:5000},"target_img",true)

Export.image.toDrive({
  image: target_img.select(bands),
  description: file_name, 
  scale: 10,
  maxPixels:10000000000,
  folder: folder_name,
  region: region_A,
  fileFormat: 'GeoTIFF'
});

In [ ]:
# 导出影像：
# 使用Export.image.toDrive方法将目标影像（target_img）导出到Google Drive，指定了输出波段、描述、分辨率、最大像素数、文件夹名、区域和文件格式。

In [ ]:
## 可视化
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}
m = geemap.Map()
# 定位和缩放到 Polygon 所在的区域
m.centerObject(anyue_boundaries, zoom=8)  # 可以调整 zoom 级别以适合你的需求
m.add_layer(anyue_image, visualization, 'RGB')
m.addLayer(anyue_boundaries, {'color': 'FF0000'}, 'Polygon Layer')
m

Map(center=[29.996204967109406, 105.39182801661268], controls=(WidgetControl(options=['position', 'transparent…